This is the Glob-style path to the subfolder containing niftis of interest
- For example, from the base_directory, */tissue_segment_z_scores will look for all subjects, all session folders within subjects, and then check the tissue_segment_z_scores folder. 


In [1]:
base_directory = '/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/resources/datasets/DBS_studies_influencing_memory/derivatives'
file_pattern = 'sub-*/ses-*/roi/sub-*.0mm_sphere_roi_composite.nii.gz'

In [2]:
from calvin_utils.file_utils.import_functions import GiiNiiFileImport
dv_df = GiiNiiFileImport(import_path=base_directory, file_pattern=file_pattern).run()
dv_df

Attempting to import from: /Users/cu135/Partners HealthCare Dropbox/Calvin Howard/resources/datasets/DBS_studies_influencing_memory/derivatives/sub-*/ses-*/roi/sub-*.0mm_sphere_roi_composite.nii.gz


,/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/resources/datasets/DBS_studies_influencing_memory/derivatives/sub-Hansen_2018/ses-ERC_BL/roi/sub-Hansen_2018_MNI152_T1_1.0mm_sphere_roi_composite.nii.gz,/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/resources/datasets/DBS_studies_influencing_memory/derivatives/sub-Inman_2018/ses-AMY_L/roi/sub-Inman_2018_MNI152_T1_1.0mm_sphere_roi_composite.nii.gz,/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/resources/datasets/DBS_studies_influencing_memory/derivatives/sub-Mankin_2021/ses-ERC_R/roi/sub-Mankin_2021_MNI152_T1_1.0mm_sphere_roi_composite.nii.gz,/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/resources/datasets/DBS_studies_influencing_memory/derivatives/sub-Halgren_1985/ses-HPC_&_AMY_BL/roi/sub-Halgren_1985_MNI152_T1_1.0mm_sphere_roi_composite.nii.gz,/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/resources/datasets/DBS_studies_influencing_memory/derivatives/sub-Turnbul_1985/ses-BNM_BL/roi/sub-Turnbul_1985_MNI152_T1_1.0mm_sphere_roi_composite.nii.gz,/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/resources/datasets/DBS_studies_influencing_memory/derivatives/sub-Hampson_2018/ses-HPC_BL_L_R/roi/sub-Hampson_2018_MNI152_T1_1.0mm_sphere_roi_composite.nii.gz,/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/resources/datasets/DBS_studies_influencing_memory/derivatives/sub-Oh_2012/ses-ANT_BL/roi/sub-Oh_2012_MNI152_T1_1.0mm_sphere_roi_composite.nii.gz,/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/resources/datasets/DBS_studies_influencing_memory/derivatives/sub-Velasco_1987/ses-CM_BL/roi/sub-Velasco_1987_MNI152_T1_1.0mm_sphere_roi_composite.nii.gz,/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/resources/datasets/DBS_studies_influencing_memory/derivatives/sub-Velasco_1987/ses-Fornix_L/roi/sub-Velasco_1987_MNI152_T1_1.0mm_sphere_roi_composite.nii.gz,/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/resources/datasets/DBS_studies_influencing_memory/derivatives/sub-Lin_2020/ses-Fornix_BL/roi/sub-Lin_2020_MNI152_T1_1.0mm_sphere_roi_composite.nii.gz,...,/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/resources/datasets/DBS_studies_influencing_memory/derivatives/sub-Coleshill_1990/ses-HPC_L/roi/sub-Coleshill_1990_MNI152_T1_1.0mm_sphere_roi_composite.nii.gz,/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/resources/datasets/DBS_studies_influencing_memory/derivatives/sub-Coleshill_1990/ses-HPC_R/roi/sub-Coleshill_1990_MNI152_T1_1.0mm_sphere_roi_composite.nii.gz,/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/resources/datasets/DBS_studies_influencing_memory/derivatives/sub-Asahi_2014/ses-STN_BL/roi/sub-Asahi_2014_MNI152_T1_1.0mm_sphere_roi_composite.nii.gz,/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/resources/datasets/DBS_studies_influencing_memory/derivatives/sub-Laxton_2010/ses-Fornix_BL/roi/sub-Laxton_2010_MNI152_T1_1.0mm_sphere_roi_composite.nii.gz,/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/resources/datasets/DBS_studies_influencing_memory/derivatives/sub-Wang_2022/ses-STN_BL/roi/sub-Wang_2022_MNI152_T1_1.0mm_sphere_roi_composite.nii.gz,/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/resources/datasets/DBS_studies_influencing_memory/derivatives/sub-Williams_2011/ses-STN_BL/roi/sub-Williams_2011_MNI152_T1_1.0mm_sphere_roi_composite.nii.gz,/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/resources/datasets/DBS_studies_influencing_memory/derivatives/sub-Salanova_2021/ses-ANT_BL/roi/sub-Salanova_2021_MNI152_T1_1.0mm_sphere_roi_composite.nii.gz,/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/resources/datasets/DBS_studies_influencing_memory/derivatives/sub-Zhang_2021/ses-BNM_BL/roi/sub-Zhang_2021_MNI152_T1_1.0mm_sphere_roi_composite.nii.gz,/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/resources/datasets/DBS_studies_influencing_memory/derivatives/sub-Dujardin_2001/ses-STN_BL/roi/sub-Dujardin_2001_MNI152_T1_1.0mm_sphere_roi_composite.nii.gz,/Users/cu135/Partner

**Extract Subject ID From File Names**
Using the example filenames that have been printed above, please define a general string:
1) Preceding the subject ID.
2) Proceeding the subject ID. 

This Should Often Be Left Default

In [3]:
# preceding_id = '/sub-'
# proceeding_id = ''

In [4]:
# from calvin_utils.file_utils.import_functions import GiiNiiFileImport
# dv_df = GiiNiiFileImport.splice_colnames(dv_df, preceding_id, proceeding_id)
# dv_df

# Load Map to Check Damage Within

Import Region of Interest Masks

In [5]:
base_directory = '/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/studies/ccm_memory/results/notebook_07/best_maps/weighted_average'
shared_glob_pattern = 'all_datasets_weighted_avg_r_map_pcc_inverse_r_map.nii.gz'

In [6]:
from calvin_utils.file_utils.import_functions import GiiNiiFileImport
iv_df = GiiNiiFileImport(import_path=base_directory, file_pattern=shared_glob_pattern).run()
iv_df

Attempting to import from: /Users/cu135/Partners HealthCare Dropbox/Calvin Howard/studies/ccm_memory/results/notebook_07/best_maps/weighted_average/all_datasets_weighted_avg_r_map_pcc_inverse_r_map.nii.gz


,/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/studies/ccm_memory/results/notebook_07/best_maps/weighted_average/all_datasets_weighted_avg_r_map_pcc_inverse_r_map.nii.gz
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0
...,...
902624,0.0
902625,0.0
902626,0.0
902627,0.0


Extract Damage Scores Per Region of Interest

In [7]:
mask_path = '/Users/cu135/hires_backdrops/MNI/MNI152_T1_2mm_brain_mask.nii'

In [8]:
from calvin_utils.nifti_utils.damage_score_utils import DamageScorer
damage_scorer = DamageScorer(mask_path, dv_df, iv_df)
dmg_df = damage_scorer.calculate_damage_scores()
dmg_df = damage_scorer.sort_dataframes_by_index(dmg_df)
dmg_df

,all_datasets_weighted_avg_r_map_pcc_inverse_r_map_spatial_corr,all_datasets_weighted_avg_r_map_pcc_inverse_r_map_cosine,all_datasets_weighted_avg_r_map_pcc_inverse_r_map_sum,all_datasets_weighted_avg_r_map_pcc_inverse_r_map_average_subject_in_target,all_datasets_weighted_avg_r_map_pcc_inverse_r_map_average_target_in_subject,all_datasets_weighted_avg_r_map_pcc_inverse_r_map_num_in_roi
path,,,,,,
/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/resources/datasets/DBS_studies_influencing_memory/derivatives/sub-Hansen_2018/ses-ERC_BL/roi/sub-Hansen_2018_MNI152_T1_1.0mm_sphere_roi_composite.nii.gz,-0.003128,-0.003145,-0.665026,-4.986102e-06,-0.332513,0.0
/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/resources/datasets/DBS_studies_influencing_memory/derivatives/sub-Inman_2018/ses-AMY_L/roi/sub-Inman_2018_MNI152_T1_1.0mm_sphere_roi_composite.nii.gz,0.001887,0.001870,0.395352,2.964191e-06,0.197676,0.0
/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/resources/datasets/DBS_studies_influencing_memory/derivatives/sub-Mankin_2021/ses-ERC_R/roi/sub-Mankin_2021_MNI152_T1_1.0mm_sphere_roi_composite.nii.gz,-0.000336,-0.000348,-0.052048,-3.902361e-07,-0.052048,0.0
/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/resources/datasets/DBS_studies_influencing_memory/derivatives/sub-Halgren_1985/ses-HPC_&_AMY_BL/roi/sub-Halgren_1985_MNI152_T1_1.0mm_sphere_roi_composite.nii.gz,-0.008164,-0.008192,-3.000547,-2.249690e-05,-0.500091,0.0
/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/resources/datasets/DBS_studies_influencing_memory/derivatives/sub-Turnbul_1985/ses-BNM_BL/roi/sub-Turnbul_1985_MNI152_T1_1.0mm_sphere_roi_composite.nii.gz,0.001664,0.001647,0.348254,2.611072e-06,0.174127,0.0
...,...,...,...,...,...,...
/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/resources/datasets/DBS_studies_influencing_memory/derivatives/sub-Williams_2011/ses-STN_BL/roi/sub-Williams_2011_MNI152_T1_1.0mm_sphere_roi_composite.nii.gz,0.003904,0.003887,0.822004,6.163058e-06,0.411002,0.0
/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/resources/datasets/DBS_studies_influencing_memory/derivatives/sub-Salanova_2021/ses-ANT_BL/roi/sub-Salanova_2021_MNI152_T1_1.0mm_sphere_roi_composite.nii.gz,0.005996,0.005979,1.264328,9.479430e-06,0.632164,0.0
/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/resources/datasets/DBS_studies_influencing_memory/derivatives/sub-Zhang_2021/ses-BNM_BL/roi/sub-Zhang_2021_MNI152_T1_1.0mm_sphere_roi_composite.nii.gz,0.001664,0.001647,0.348254,2.611072e-06,0.174127,0.0


Save the Results

In [9]:
analysis = ''
session = 'dbs'
root_dir = '/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/resources/datasets/BIDS_AD_DBS_FORNIX/study_metadata/study_metadata/ccm_study/metadata.csv'

In [10]:
# damage_scorer.save_csv_to_metadata(dmg_df, root_dir=root_dir, analysis=analysis, ses=session, dry_run=False)

Save Results into Master CSV

In [11]:
master_path = '/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/resources/datasets/DBS_studies_influencing_memory/metadata/master_list.csv'
sheet = None

In [12]:
from calvin_utils.permutation_analysis_utils.statsmodels_palm import CalvinStatsmodelsPalm
cal_palm = CalvinStatsmodelsPalm(input_csv_path=master_path, output_dir=None, sheet=sheet)
# Call the process_nifti_paths method
data_df = cal_palm.read_and_display_data()
data_df

,study,Memory_Specific,ses,N,Memory_Outcome,Pre_Post_Memory_Effect_Size__Cohen_s_D_,Location,List_of_Coordinates,Side,Frequency__Hz_,...,all_datasets_weighted_avg_r_map_inverted_sum,all_datasets_weighted_avg_r_map_inverted_average_subject_in_target,all_datasets_weighted_avg_r_map_inverted_average_target_in_subject,all_datasets_weighted_avg_r_map_inverted_num_in_roi,all_datasets_unweighted_avg_r_map_pcc_inverse_r_map_spatial_corr,all_datasets_unweighted_avg_r_map_pcc_inverse_r_map_cosine,all_datasets_unweighted_avg_r_map_pcc_inverse_r_map_sum,all_datasets_unweighted_avg_r_map_pcc_inverse_r_map_average_subject_in_target,all_datasets_unweighted_avg_r_map_pcc_inverse_r_map_average_target_in_subject,all_datasets_unweighted_avg_r_map_pcc_inverse_r_map_num_in_roi
0,Acera 2019,0.0,STN_BL,50.0,Impaired,-0.180960,STN,"[14.2, -15.1, -6.1], [-13.5, -14.2, -6.1]",BL,NaN,...,-0.510350,-3.999358e-06,-0.255175,0.0,-0.000956,-0.000988,-0.324409,-0.000002,-0.162204,0.0
1,Asahi 2014,0.0,STN_BL,11.0,Improved,0.234145,STN,"[-16.7, -6.0, -1.7], [18.6, -5.3, -1.7]",BL,130.0,...,-0.134679,-1.055413e-06,-0.067340,0.0,-0.002844,-0.002876,-0.944059,-0.000007,-0.472029,0.0
2,Barcia 2022,1.0,Fornix_BL,1.0,Mixed,NaN,Fornix,"[-2, 3.5, -1], [2, 3.5, -1]",BL,NaN,...,0.496349,3.889635e-06,0.248174,0.0,0.002752,0.002718,0.892096,0.000007,0.446048,0.0
3,Bergfeld 2017,0.0,vALIC_BL,20.0,Impaired,-0.688208,vALIC,"[-11.6, 7.2, -3.2], [12.7, 8.7, -2.4]",BL,140.0,...,0.017347,1.359416e-07,0.008674,0.0,0.001610,0.001577,0.517480,0.000004,0.258740,0.0
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.134679,-1.055413e-06,-0.067340,0.0,-0.002844,-0.002876,-0.944059,-0.000007,-0.472029,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.510350,-3.999358e-06,-0.255175,0.0,-0.000956,-0.000988,-0.324409,-0.000002,-0.162204,0.0
101,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.134679,-1.055413e-06,-0.067340,0.0,-0.002844,-0.002876,-0.944059,-0.000007,-0.472029,0.0
102,You 2020,0.0,STN_BL,NaN,NaN,NaN,STN,"[-16.7, -6.0, -1.7], [18.6, -5.3, -1.7]",BL,NaN,...,-0.134679,-1.055413e-06,-0.067340,0.0,-0.002844,-0.002876,-0.944059,-0.000007,-0.472029,0.0
103,Zangaglia 2009,0.0,STN_BL,32.0,Improved,0.099419,STN,"[-16.7, -6.0, -1.7], [18.6, -5.3, -1.7]",BL,NaN,...,-0.134679,-1.055413e-06,-0.067340,0.0,-0.002844,-0.002876,-0.944059,-0.000007,-0.472029,0.0


In [13]:
# Perform an outer join on dmg_df and data_df using the 'path' column
merged_df = data_df.merge(dmg_df, on='path', how='outer')
merged_df

,study,Memory_Specific,ses,N,Memory_Outcome,Pre_Post_Memory_Effect_Size__Cohen_s_D_,Location,List_of_Coordinates,Side,Frequency__Hz_,...,all_datasets_unweighted_avg_r_map_pcc_inverse_r_map_sum,all_datasets_unweighted_avg_r_map_pcc_inverse_r_map_average_subject_in_target,all_datasets_unweighted_avg_r_map_pcc_inverse_r_map_average_target_in_subject,all_datasets_unweighted_avg_r_map_pcc_inverse_r_map_num_in_roi,all_datasets_weighted_avg_r_map_pcc_inverse_r_map_spatial_corr,all_datasets_weighted_avg_r_map_pcc_inverse_r_map_cosine,all_datasets_weighted_avg_r_map_pcc_inverse_r_map_sum,all_datasets_weighted_avg_r_map_pcc_inverse_r_map_average_subject_in_target,all_datasets_weighted_avg_r_map_pcc_inverse_r_map_average_target_in_subject,all_datasets_weighted_avg_r_map_pcc_inverse_r_map_num_in_roi
0,Acera 2019,0.0,STN_BL,50.0,Impaired,-0.180960,STN,"[14.2, -15.1, -6.1], [-13.5, -14.2, -6.1]",BL,NaN,...,-0.324409,-0.000002,-0.162204,0.0,0.003904,0.003887,0.822004,0.000006,0.411002,0.0
1,Asahi 2014,0.0,STN_BL,11.0,Improved,0.234145,STN,"[-16.7, -6.0, -1.7], [18.6, -5.3, -1.7]",BL,130.0,...,-0.944059,-0.000007,-0.472029,0.0,0.002230,0.002213,0.467976,0.000004,0.233988,0.0
2,Barcia 2022,1.0,Fornix_BL,1.0,Mixed,NaN,Fornix,"[-2, 3.5, -1], [2, 3.5, -1]",BL,NaN,...,0.892096,0.000007,0.446048,0.0,-0.003976,-0.003993,-0.844374,-0.000006,-0.422187,0.0
3,Bergfeld 2017,0.0,vALIC_BL,20.0,Impaired,-0.688208,vALIC,"[-11.6, 7.2, -3.2], [12.7, 8.7, -2.4]",BL,140.0,...,0.517480,0.000004,0.258740,0.0,0.002615,0.002598,0.549454,0.000004,0.274727,0.0
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.944059,-0.000007,-0.472029,0.0,0.002230,0.002213,0.467976,0.000004,0.233988,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.324409,-0.000002,-0.162204,0.0,0.003904,0.003887,0.822004,0.000006,0.411002,0.0
101,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.944059,-0.000007,-0.472029,0.0,0.002230,0.002213,0.467976,0.000004,0.233988,0.0
102,You 2020,0.0,STN_BL,NaN,NaN,NaN,STN,"[-16.7, -6.0, -1.7], [18.6, -5.3, -1.7]",BL,NaN,...,-0.944059,-0.000007,-0.472029,0.0,0.002230,0.002213,0.467976,0.000004,0.233988,0.0
103,Zangaglia 2009,0.0,STN_BL,32.0,Improved,0.099419,STN,"[-16.7, -6.0, -1.7], [18.6, -5.3, -1.7]",BL,NaN,...,-0.944059,-0.000007,-0.472029,0.0,0.002230,0.002213,0.467976,0.000004,0.233988,0.0


In [14]:
merged_df.to_csv(master_path, index=False)

All done. Enjoy your analyses. 

--Calvin 